In [4]:
from rdflib import Namespace, Graph, URIRef, Literal, XSD
import json

from get_schema_predicate import return_predicate

In [5]:
file_path = "/Users/heede/Library/CloudStorage/OneDrive-AalborgUniversitet/DVML P7/Data/yelp_academic_dataset_business.json"

In [19]:
schema = Namespace("https://schema.org/")

yelp_business = Namespace("https://www.yelp.com/biz/")
yelp_user = Namespace("https://www.yelp.com/user_details?userid=")
yelp_review_vc = Namespace("https://www.yelp.com/kg/review/votes_count/")

In [20]:
G = Graph()

In [21]:
with open(file=file_path, mode="r") as file:
    for line in file:
        line = json.loads(line)

        json_key = list(line.keys())[0]
        _subject = line[json_key]
        del line[json_key]
        print(json_key, _subject)

        for _predicate, _object in line.items():
            if isinstance(_object, dict):
                pass
            else:
                predicate, object_type = return_predicate(_predicate)
                G.add(triple=(URIRef(yelp_business + _subject), # Subject
                              URIRef(schema + predicate), # Predicate
                              Literal(_object, datatype=object_type) # Object
                      ))


business_id Pns2l4eNsfO8kk83dixA6A


AssertionError: Predicate has_Name must be an rdflib term

In [36]:
dct = dict()
with open(file=file_path, mode="r") as file:
    for line in file:
        line = json.loads(line)
        # print(line["attributes"])
        #
        if isinstance(line["attributes"], dict):
            dct |= line["attributes"]

In [39]:
dct

{'ByAppointmentOnly': 'False',
 'BusinessAcceptsCreditCards': 'True',
 'BikeParking': 'False',
 'RestaurantsPriceRange2': '1',
 'CoatCheck': 'False',
 'RestaurantsTakeOut': 'None',
 'RestaurantsDelivery': 'None',
 'Caters': 'True',
 'WiFi': "u'free'",
 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
 'WheelchairAccessible': 'True',
 'HappyHour': 'False',
 'OutdoorSeating': 'False',
 'HasTV': 'True',
 'RestaurantsReservations': 'False',
 'DogsAllowed': 'True',
 'Alcohol': "'none'",
 'GoodForKids': 'True',
 'RestaurantsAttire': "u'casual'",
 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False}",
 'RestaurantsTableService': 'True',
 'RestaurantsGoodForGroups': 'True',
 'DriveThru': 'True',
 'NoiseLevel': "u'average'",
 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'break